In [1]:
import sys,os,glob,copy
import numpy as np
import pyslha
import pandas as pd
import ROOT
import seaborn as sns
from collections import namedtuple
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d,griddata,LinearNDInterpolator
sys.path.append('../ATLAS_data')
from effFunctions import (getMuonRecoEff,getTriggerEff,getTrackEff,
                                     getSelectionEff,getTargetMass,getMassSelEff)

# os.environ['ROOT_INCLUDE_PATH'] = os.path.abspath('./MG5/Delphes/external')

ROOT.gSystem.Load(os.path.abspath("../DelphesHSCP/libDelphes.so"))


ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

plt.rcParams.update({
    "figure.figsize" : (8,6),
    "lines.linewidth" : 3.0,
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

Welcome to JupyROOT 6.26/04


In [2]:
inputFiles = {'10ns' : ['../Wino_0j_match/Events/run_88/wino_1300_10ns_delphes_events.root',
                       '../Wino_1j_match/Events/run_01/wino_1300_10ns_delphes_events.root'] }
# xsecsPB = {'stable' : 7.133e-3, 'prompt' : 7.133e-3}

In [3]:
modelInfoDict = {}
for label,f in inputFiles.items():
    f = f[0]
    if not os.path.isfile(f):
        print('File %s not found' %f)
        raise OSError()
    parsDict = {}    
    for banner in glob.glob(os.path.join(os.path.dirname(f),'*banner*txt')):
        with open(banner,'r') as ff:
            slhaData = ff.read().split('<slha>')[1].split('</slha>')[0]
            slhaData = pyslha.readSLHA(slhaData)
    parsDict = {}
    parsDict['mLLP'] = slhaData.blocks['MASS'][1000024]
    parsDict['mLSP'] = slhaData.blocks['MASS'][1000022]
    parsDict['width'] = slhaData.decays[1000024].totalwidth
    if parsDict['width']:
        parsDict['tau_ns'] = (6.582e-25/parsDict['width'])*1e9
    else:
        parsDict['tau_ns'] = np.inf    
    modelInfo = namedtuple('modelInfo', ' '.join(parsDict.keys()))
    m = modelInfo(**parsDict)
    modelInfoDict[label] = m
    print(label,':')
    print('mLLP = ',m.mLLP)
    print('tau (ns) = ',m.tau_ns)

10ns :
mLLP =  1300.0
tau (ns) =  9.999999999999998


### Define useful functions

In [4]:
def getHSCPCandidates(hscps,daughters):

    candidates = []
    for ip in range(hscps.GetEntries()):
        p = hscps.At(ip)
        if p.Status == 1:
            p.r_decay = np.inf
            p.z_decay = np.inf
            candidates.append(p)
            continue
        
        # If HSCP is unstable compute its decay position from daughter
        d1 = p.D1
        daughter = daughters.At(d1)
        # Get the HSCP decay radius from the first daughter production vertex:
        p.r_decay = np.sqrt(daughter.X**2+daughter.Y**2)      
        p.z_decay = daughter.Z        
        candidates.append(p)

    for p in candidates:
        # Add beta and gamma*beta
        trimom = np.sqrt(p.Px**2 +p.Py**2 + p.Pz**2)
        p.beta = trimom/p.E
        p.gbeta = trimom/p.Mass

    return candidates

def applyHSCPSelection(hscpList,pT=50.,eta=2.4,r=500.):

    selHSCPs = []
    for hscp in hscpList:
        if hscp.PT < pT: continue 
        if abs(hscp.Eta) > eta: continue
        if hscp.r_decay < r: continue
        selHSCPs.append(hscp)
    
    return selHSCPs

def applyIsolation(hscpList,tracks):

    isoHSCPs = []
    # Apply isolation requirement for HSCP tracks
    for hscp in hscpList:
        sumPT = 0.0
        for itrk in range(tracks.GetEntries()):
            track = tracks.At(itrk)
            if abs(track.PID) > 10000:
                continue
            dPhi =(track.Phi-hscp.Phi)
            dPhi = np.arctan2(np.sin(dPhi),np.cos(dPhi))
            deltaR = np.sqrt(dPhi**2 +(track.Eta-hscp.Eta)**2)
            if deltaR > 0.3:
                continue
            sumPT += track.PT
        if sumPT > 5.0: continue
        isoHSCPs.append(hscp)
    return isoHSCPs


def applyMuonTagging(hscpList):

    """
    Computes the probability of reconstructing the hscp as a muon.
    :param hscpList: List of GenParticle objects
    """

    muonsLLP = []
    for hscp in hscpList:
        if hscp.r_decay < 3.9e3 and hscp.z_decay < 6e3: # Skip decays before MS
            continue
        
        beta = hscp.beta
        eta = abs(hscp.Eta)
        eff = getMuonRecoEff(beta,eta,hscp.PID)
        # Randomly reconstrunct the HSCP as a muon
        if np.random.uniform() < eff:
            continue
        muonsLLP.append(hscp)
    
    return muonsLLP

def removeFromMET(particles,METobj):
    """
    Removes the contribution from the particles in the list
    to the total MET.
    """

    metx = METobj.MET*np.cos(METobj.Phi)
    mety = METobj.MET*np.sin(METobj.Phi)

    if particles:
        # Remove particles from MET:            
        pxTot = sum([p.Px for p in particles])
        pyTot = sum([p.Py for p in particles])        
        metx = (metx-pxTot)
        mety = (mety-pyTot)
    
    return [metx,mety]

def applyMTCut(hscps,METvector):
    """
    Remove tracks which have mT < 130 GeV
    """

    selHSCPs = []
    met = np.sqrt(METvector[0]**2 + METvector[1]**2)    
    for hscp in hscps:
        pThscp = [hscp.Px,hscp.Py]
        cosdphi = np.dot(pThscp,METvector)/(hscp.PT*met)
        mT = np.sqrt(2*hscp.PT*met*(1-cosdphi))
        if mT < 130.: continue
        selHSCPs.append(hscp)
    
    return selHSCPs


### Loop over events, apply basic selection criterium and compute masses

In [5]:
weightsDict = {}
massesDict = {}
nhscps = {}
gbetaDict = {}
trackEffDict = {}
metDict = {}
newMetDict = {}
metCaloDict = {}
genMetDict = {}
cutFlowDict = {}

for label,inputFileList in inputFiles.items():

    modelPars = modelInfoDict[label]

    
    gbetaDict[label] = []
    nhscps[label] = []
    trackEffDict[label] = []
    metDict[label] = []
    newMetDict[label] = []
    metCaloDict[label] = []
    genMetDict[label] = []
    weightsDict[label] = []
    isolation = []
    rdecay = []
    totalWeight = 0.0
    srWeights = {}
    
    cutFlow = { "Total" : 0.0,
                "Trigger" : 0.0,
                "ETmiss > 170 GeV" : 0.0,
                "pT > 50 GeV" : 0.0,
                "Track isolation" : 0.0,
                "pT > 120 GeV" : 0.0,
                "eta < 1.8" : 0.0,
                "mT(track,pTmiss) > 130 GeV" : 0.0,
                "(Acceptance)" : 0.0,
                "(SR-Low - no mass Window)" : 0.0,
                "(SR-High - no mass Window)" : 0.0          
                }

    weights = []
    masses = []
    
    for inputFile in inputFileList:    
        
        f = ROOT.TFile(inputFile,'read')
        tree = f.Get("Delphes")
        nevts = tree.GetEntries()

        # nevts = 100
        print('Reading %i events' %nevts)
        for ievt in range(nevts):
            tree.GetEntry(ievt)
            weight = tree.Weight.At(1).Weight
            totalWeight += weight
            metCalo = tree.MissingETCalo.At(0).MET
            hscpCandidates = getHSCPCandidates(tree.isoHSCPs,tree.hscpDaughters)
            dmParticles = [tree.dmParticles.At(idm) for idm in range(tree.dmParticles.GetEntries())]
            nhscps[label].append(len(hscpCandidates))
            if not hscpCandidates:
                continue

            # For chargino -> LSP + pion the LSP+LSP pT is ≃ the gaugino system (C1N1 or C1C1) pT
            gauginoPT = sum([p.Px for p in dmParticles])**2
            gauginoPT += sum([p.Py for p in dmParticles])**2
            gauginoPT = np.sqrt(gauginoPT)
            # Apply pT cut on HSCP system to reproduce ATLAS selection
            if gauginoPT < 60.0:
                continue

            hscpsFilter = applyHSCPSelection(hscpCandidates,pT=120.,eta=1.8,r=500.0)
            if hscpsFilter:
                cutFlow['(Acceptance)'] += weight

            muonsLLP = applyMuonTagging(hscpCandidates)
            hscps = [hscp for hscp in hscpCandidates if hscp not in muonsLLP]
            newMETv = removeFromMET(muonsLLP,tree.MissingET.At(0))
            newMET = np.sqrt(newMETv[0]**2+newMETv[1]**2)
            triggerEff = getTriggerEff(metCalo)
            
            cutFlow['Total'] += weight
            weight = weight*triggerEff
            if not weight: continue
            cutFlow['Trigger'] += weight
            # Apply event selection efficiency
            eventEff = getSelectionEff(newMET)
            weight = weight*eventEff
            if not weight: continue
            cutFlow["ETmiss > 170 GeV"] += weight

            # Apply selection to HSCP candidates (following ATLAS snippet)
            hscps = applyHSCPSelection(hscps,pT=50.,eta=3.0,r=500.0)
            if not hscps: continue
            cutFlow['pT > 50 GeV'] += weight        
            # hscps = applyIsolation(hscps,tree.Track) # Already included in trackEff
            if not hscps: continue
            cutFlow['Track isolation'] += weight     
            hscps = applyHSCPSelection(hscps,pT=120.)
            if not hscps: continue
            cutFlow['pT > 120 GeV'] += weight  
            if not hscps: continue
            hscps = applyHSCPSelection(hscps,pT=120.,eta=1.8)
            if not hscps: continue
            cutFlow['eta < 1.8'] += weight
#             hscps = applyMTCut(hscps,newMETv) # Already included in trackEff
            if not hscps: continue
            cutFlow['mT(track,pTmiss) > 130 GeV'] += weight


            # Select hscps which fall into one of the mass windows:
            # htargetMass = [(hscp,getTargetMass(hscp.Mass)) for hscp in hscps]
            # htargetMass = [x for x in htargetMass[:] if x[1] is not None]
            # hscps = [x[0] for x in htargetMass]
            gbetas = [h.gbeta for h in hscps]
            # targetMass = [x[1] for x in htargetMass]
            targetMass = [h.Mass for h in hscps]
            trackEffHigh = getTrackEff(gbetas,sr='High')
            trackEffLow =  getTrackEff(gbetas,sr='Low')
            wMassHigh = getMassSelEff(targetMass,sr='High')
            wMassLow = getMassSelEff(targetMass,sr='Low')
            eventWeightLow = weight*(1-np.prod(1.0-trackEffLow*wMassLow))
            eventWeightHigh = weight*(1-np.prod(1.0-trackEffHigh*wMassHigh))

            cutFlow['(SR-High - no mass Window)'] += weight*(1-np.prod(1.0-trackEffHigh))
            cutFlow['(SR-Low - no mass Window)'] += weight*(1-np.prod(1.0-trackEffLow))

            gbetaDict[label] += gbetas
            trackEffDict[label] += trackEffLow.tolist()

            genMetDict[label].append(tree.GenMissingET.At(0).MET)
            metDict[label].append(tree.MissingET.At(0).MET)
            newMetDict[label].append(newMET)
            
            # Use maximum mass to select final mass window
            targetMass = max(targetMass)
            if not targetMass in srWeights:
                srWeights[targetMass] = {'Low' : 0.0, 'High' : 0.0}
            srWeights[targetMass]['Low'] += eventWeightLow
            srWeights[targetMass]['High'] += eventWeightHigh
        
        cutFlowDict[label] = cutFlow
        f.Close()
        
    print("%s :" %label)
    print('Total cross-section = %1.3e pb' %totalWeight)
    print('Total cross-section (pT > 60 GeV) = %1.3g pb' %cutFlow['Total'])
    print('SR=',srWeights)

Reading 38252 events
Reading 29768 events
10ns :
Total cross-section = 2.612e-04 pb
Total cross-section (pT > 60 GeV) = 0.00013 pb
SR= {1300.0: {'Low': 2.6407808005667167e-06, 'High': 7.990865057764568e-06}}


In [6]:
for m in srWeights:
    print('Ns (Low) = %1.3f' %(srWeights[m]['Low']*1e3*139))
    print('Ns (High) = %1.3f' %(srWeights[m]['High']*1e3*139))

Ns (Low) = 0.367
Ns (High) = 1.111


In [10]:
lumi = 139.0
for labe,cutFlow in cutFlowDict.items():
    print('  %s: '%label.rjust(30),'Events'.rjust(8),'(Efficiency)'.rjust(16))
    for c,val in cutFlow.items():
        print('   %s'%c.rjust(30),' = %8.3e'%(val*1e3*139),' (%1.3f)' %(val/cutFlow['Total']))

                            10ns:    Events     (Efficiency)
                            Total  = 1.811e+01  (1.000)
                          Trigger  = 1.046e+01  (0.577)
                 ETmiss > 170 GeV  = 7.198e+00  (0.397)
                      pT > 50 GeV  = 4.745e+00  (0.262)
                  Track isolation  = 4.745e+00  (0.262)
                     pT > 120 GeV  = 4.667e+00  (0.258)
                        eta < 1.8  = 4.377e+00  (0.242)
       mT(track,pTmiss) > 130 GeV  = 4.377e+00  (0.242)
                     (Acceptance)  = 1.141e+01  (0.630)
        (SR-Low - no mass Window)  = 6.063e-01  (0.033)
       (SR-High - no mass Window)  = 1.625e+00  (0.090)


In [8]:
# Debug info:
# ATLAS values for 10ns charginos with mass = 1300 GeV
# (S95_Low = 15, S95_High = 4.2, totalXsec = 2.85e-2 pb, xsecUL = 1.52e-3 pb, lumi = 139/fb)
acceptance = cutFlow['(Acceptance)']/cutFlow['Total']
evtEff = (cutFlow['ETmiss > 170 GeV']/cutFlow['(Acceptance)'])
effLow = (cutFlow['(SR-Low - no mass Window)']/cutFlow['(Acceptance)'])
effHigh = (cutFlow['(SR-High - no mass Window)']/cutFlow['(Acceptance)'])
print('Mass = %1.1f GeV, lifetime = %1.2f ns:' %(modelPars.mLLP,modelPars.tau_ns)) 
print('\t acceptance = %1.3f' %(acceptance)) # 0.634
print('\t event-level efficiency = %1.3f' %(evtEff)) # 0.395
print('\t efficiency for SR-Inclusive_Low = %1.3f' %(effLow)) # 0.061
print('\t efficiency for SR-Inclusive_High = %1.3f' %(effHigh)) # 0.146


Mass = 1300.0 GeV, lifetime = 10.00 ns:
	 acceptance = 0.630
	 event-level efficiency = 0.631
	 efficiency for SR-Inclusive_Low = 0.053
	 efficiency for SR-Inclusive_High = 0.142


### Check ATLAS official values

In [9]:
charginoData = pd.read_pickle('./ATLAS_data/charginoData.pcl')
vals = charginoData[(charginoData['mass (GeV)'] == round(modelPars.mLLP,-2)) 
                    & (charginoData['lifetime (ns)'] == round(modelPars.tau_ns,0))]
vals.T

FileNotFoundError: [Errno 2] No such file or directory: './ATLAS_data/charginoData.pcl'

In [ ]:
bins = np.linspace(0.,2000.,30)
fig = plt.figure(figsize=(8,6))
plt.hist(genMetDict['10ns'],bins=bins,label='MET (Truth)',histtype='step',linewidth=3)
plt.hist(metDict['10ns'],bins=bins,label='MET (Total)',histtype='step',linewidth=3)
plt.hist(newMetDict['10ns'],bins=bins,label='MET (New)',histtype='step',linewidth=3,linestyle='--')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
bins = np.arange(0.,4.,0.0666)
plt.hist(gbetaDict['10ns'],bins=bins,histtype='step',linewidth=3)
plt.title(r'$\langle \gamma \beta \rangle = %1.2f$' %np.mean(gbetaDict['10ns']))
plt.xlabel(r'$\gamma \beta$')
plt.vlines(x=0.3,ymin=0.,ymax=1000,linestyles='--',color='gray')
plt.vlines(x=0.9,ymin=0.,ymax=1000,linestyles='--',color='gray')
plt.grid()
plt.minorticks_on()
plt.xlim(0.,4.)
plt.show()

In [ ]:
plt.scatter(gbetaDict['10ns'],trackEffDict['10ns'])
plt.xlim(0.5,2.0)
plt.grid()
plt.title(r'$\langle \epsilon_{trk} \rangle = %1.2f \pm %1.2f$' %(np.mean(trackEffDict['10ns']),np.std(trackEffDict['10ns'])))
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.hist(trackEffDict['10ns'],bins=50,histtype='step',linewidth=3)
plt.title(r'$\langle \epsilon_{trk} \rangle = %1.2f \pm %1.2f$' %(np.mean(trackEffDict['10ns']),np.std(trackEffDict['10ns'])))
plt.xlabel(r'$\epsilon_{trk}$')
plt.grid()
plt.yscale('log')
plt.minorticks_on()
# plt.xlim(0.,4.)
plt.show()

In [ ]:
plt.hist(nhscps['10ns'],bins=30)
plt.show()

In [ ]:
np.count_nonzero(np.array(nhscps['10ns']) == 1),np.count_nonzero(np.array(nhscps['10ns']) == 2)